<a href="https://colab.research.google.com/github/rahimnadan/fashion_mnist_dataset_CNN_Optimized_model_using_keras_tuner/blob/main/Creating_a_CNN_model_with_optimization_using_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing the library keras_tuner
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.1 MB/s eta 0:00:00


In [5]:
# Importing the dependencies
import numpy as np
import tensorflow as tf
import keras_tuner
from keras_tuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from tensorflow import keras
from tensorflow.keras import layers



In [6]:
# importing the data set
fashion_mnist= keras.datasets.fashion_mnist

In [7]:
# Loading the data from various sources
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [8]:
#  converting the pixels to standard form

train_images= train_images / 255.0
test_images = test_images /255.0

In [9]:
train_images[0]


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [10]:
train_images[0].shape

(28, 28)

In [11]:
# Reshaping the images to standard form according to the input of the model
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [17]:
# Creating our model
def build_model(hp):
  model =keras.Sequential([
      keras.layers.Conv2D(
          filters= hp.Int('conv_1_filter', min_value = 32, max_value = 64,step=16),
          kernel_size=hp.Choice('conv_1_kernal', values= [3,5]),
          activation= 'relu',
          input_shape= (28,28,1)
      ),
      keras.layers.Conv2D(
          filters= hp.Int('conv_2_filter', min_value = 32, max_value = 64,step=16),
          kernel_size=hp.Choice('conv_2_kernal', values= [3,5]),
          activation= 'relu'

      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value = 32, max_value= 128, step=16),
          activation= 'relu'
      ),
      keras.layers.Dense(10,activation= 'softmax')

  ])

  model.compile(optimizer= keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
  loss = 'sparse_categorical_crossentropy',
  metrics = ['accuracy'])

  return model


In [18]:
# Hyperparameter tunning using keras_tuner to find the best parameters for our model

from kerastuner.engine.hyperparameters import hyperparameters

tuner_search= RandomSearch(build_model,
                           objective= 'val_accuracy',
                           max_trials = 5, directory='output',
                           project_name = 'Mnist Fashion')

tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 43s]
val_accuracy: 0.909333348274231

Best val_accuracy So Far: 0.909333348274231
Total elapsed time: 00h 03m 01s


In [19]:
# Selecting the best model
model = tuner_search.get_best_models(num_models= 1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        18464     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 64)                1179712   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,199,466
Trainable params: 1,199,466
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Retraining the model with best parameters
model.fit(train_images, train_labels, epochs=10,validation_split=0.1,initial_epoch= 3)

Epoch 4/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1507 - accuracy: 0.9435 - val_loss: 0.2794 - val_accuracy: 0.9053
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1128 - accuracy: 0.9580 - val_loss: 0.2901 - val_accuracy: 0.9117
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0804 - accuracy: 0.9697 - val_loss: 0.3446 - val_accuracy: 0.9167
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0576 - accuracy: 0.9791 - val_loss: 0.3507 - val_accuracy: 0.9133
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0418 - accuracy: 0.9851 - val_loss: 0.3875 - val_accuracy: 0.9108
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0340 - accuracy: 0.9880 - val_loss: 0.4439 - val_accuracy: 0.9202
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0284 - accuracy: 0.9900 - val_loss: 0.5089 - val_accurac

Hence Our model has been tuned with the best parameters and thus gives us the maximum accuracy.